In [1]:
# primary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# text processing libraries
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [2]:
X_train = pd.read_csv("./Dataset/cleaned_training.csv")
X_test = pd.read_csv("./Dataset/cleaned_test.csv")

X_train.drop("Unnamed: 0", axis = 1, inplace = True)
y_train = X_train['sentiment']
X_train.drop('sentiment', axis = True, inplace = True)

X_test.drop('Unnamed: 0', axis = 1, inplace = True)
y_test = X_test['sentiment']
X_test.drop("sentiment", axis = 1, inplace = True)

X_train.head()

,company,cleaned_tweets
0,Borderlands,im get borderland murder
1,Borderlands,come border kill
2,Borderlands,im get borderland kill
3,Borderlands,im come borderland murder
4,Borderlands,im get borderland murder


In [3]:
# encode target and company names
sentiment_encoder = LabelEncoder()
y_train = sentiment_encoder.fit_transform(y_train)
y_test = sentiment_encoder.transform(y_test)

company_encoder = LabelEncoder()
X_train['company'] = company_encoder.fit_transform(X_train['company'])
X_test['company'] = company_encoder.transform(X_test['company'])

X_train.head()

,company,cleaned_tweets
0,4,im get borderland murder
1,4,come border kill
2,4,im get borderland kill
3,4,im come borderland murder
4,4,im get borderland murder


In [5]:
# vectorize tweets
vec = TfidfVectorizer()
output = vec.fit_transform(X_train['cleaned_tweets'])
# X_test['cleaned_tweets'] = vec.transform(X_test['cleaned_tweets'])

X_train.head()

,company,cleaned_tweets
0,4,im get borderland murder
1,4,come border kill
2,4,im get borderland kill
3,4,im come borderland murder
4,4,im get borderland murder


In [6]:
type(output)

scipy.sparse._csr.csr_matrix